In [1]:
!mkdir dataset
!curl -L -o dataset/SOCS_Local-RFFRFS.csv "https://zenodo.org/records/10211884/files/LUCAS%20SOIL%202018%20BD%20SOCS%20Local-RFFRFS.csv?download=1" && ls -l dataset/SOCS_Local-RFFRFS.csv 
!curl -L -o dataset/SOCS_T-PTF4.csv "https://zenodo.org/records/10211884/files/LUCAS%20SOIL%202018%20BD%20SOCS%20T-PTF4.csv?download=1" && ls -l dataset/SOCS_T-PTF4.csv

A subdirectory or file dataset already exists.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1 1183k    1 17576    0     0  21343      0  0:00:56 --:--:--  0:00:56 21408
 92 1183k   92 1098k    0     0   564k      0  0:00:02  0:00:01  0:00:01  564k
100 1183k  100 1183k    0     0   601k      0  0:00:01  0:00:01 --:--:--  602k
'ls' is not recognized as an internal or external command,
operable program or batch file.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 1254k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 36 1254k   36  458k    0     0   273k      0  0:00:04  0:00:01  0:00:

In [2]:
import pandas as pd
import math

In [3]:
PATH_LOCAL = 'dataset/SOCS_Local-RFFRFS.csv'
PATH_PTF4 = 'dataset/SOCS_T-PTF4.csv'

In [4]:
data_local = pd.read_csv(PATH_LOCAL)
data_PATH_PTF4 = pd.read_csv(PATH_PTF4)

In [5]:
data_local.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15389 entries, 0 to 15388
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   POINTID          15389 non-null  int64  
 1   Bdfine (g cm-3)  15389 non-null  float64
 2   SOCS (kg cm-2)   15389 non-null  float64
 3   coarse_vol       15389 non-null  float64
 4   GPS_LAT          15389 non-null  float64
 5   GPS_LONG         15389 non-null  float64
 6   BDfine  method   15389 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 841.7+ KB


In [6]:
data_PATH_PTF4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18945 entries, 0 to 18944
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   POINTID          18945 non-null  int64  
 1   Bdfine (g cm-3)  18945 non-null  float64
 2   SOCS (kg cm-2)   18945 non-null  float64
 3   coarse_vol       18945 non-null  float64
 4   GPS_LAT          18945 non-null  float64
 5   GPS_LONG         18945 non-null  float64
 6   BDfine  method   18945 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 1.0+ MB


In [7]:
data_local.head()

,POINTID,Bdfine (g cm-3),SOCS (kg cm-2),coarse_vol,GPS_LAT,GPS_LONG,BDfine method
0,35582326,1.19,5.24,0.14,43.609317,0.544198,Measurement
1,46764408,0.96,4.35,0.05,62.617261,16.897350,Measurement
2,51464814,0.79,6.33,0.01,65.472827,27.909976,Measurement
3,29901692,1.17,2.06,0.08,37.127066,-4.962568,Measurement
4,31841902,1.06,2.77,0.02,39.318784,-3.181327,Measurement


In [8]:
data_PATH_PTF4.head()

,POINTID,Bdfine (g cm-3),SOCS (kg cm-2),coarse_vol,GPS_LAT,GPS_LONG,BDfine method
0,35582326,1.19,5.24,0.14,43.609317,0.544198,Measurement
1,46764408,0.96,4.35,0.05,62.617261,16.897350,Measurement
2,51464814,0.79,6.33,0.01,65.472827,27.909976,Measurement
3,29901692,1.17,2.06,0.08,37.127066,-4.962568,Measurement
4,31841902,1.06,2.77,0.02,39.318784,-3.181327,Measurement


In [9]:
colunas = data_local.columns
colunas

Index(['POINTID', 'Bdfine (g cm-3)', 'SOCS (kg cm-2)', 'coarse_vol', 'GPS_LAT',
       'GPS_LONG', 'BDfine  method '],
      dtype='object')

In [10]:
split_idx = int(len(data_local) * 0.7)
train = data_local.iloc[:split_idx].reset_index(drop=True)
test = data_local.iloc[split_idx:].reset_index(drop=True)
print(f"train: {len(train)} rows, test: {len(test)} rows")

train: 10772 rows, test: 4617 rows


In [11]:
column_used = 'SOCS (kg cm-2)'

In [12]:
x_train = train[colunas[0]]
y_train = train[column_used]

x_test = test[colunas[0]]
y_test = test[column_used]

In [13]:
def calculo_b1_b0(column_x,column_y):
    x_mean = column_x.sum() / len(column_x)
    y_mean = column_y.sum() / len(column_y)
    
    numerador = 0
    denominador = 0
    
    for i in range(len(column_x)):
        numerador += (column_x[i] - x_mean) * (column_y[i] - y_mean)
        denominador += (column_x[i] - x_mean) ** 2
    
    b1 = numerador / denominador
    
    b0 = y_mean - (x_mean*b1)
    
    return b1,b0
b1,b0 = calculo_b1_b0(x_train,y_train)
b1,b0

(6.582657733446041e-08, 2.300930455761806)

In [14]:
def calculo_pred(b1,b0,x):
    return (b1 * x) + b0

In [15]:
def pred_test(column_x_test):
    column_y_pred = []
    
    for x in column_x_test:
        column_y_pred.append(calculo_pred(b1,b0,x))
    
    return column_y_pred
column_y_pred = pred_test(x_test)

In [16]:
def rmse(column_y_real,column_y_pred):
    n = len(column_y_real)
    numerador = 0.0
    
    for i in range(n):
        numerador += (column_y_real[i] - column_y_pred[i]) ** 2
    
    result = numerador / n
    
    return math.sqrt(result)

rmse(y_test,column_y_pred)

6.806478723773684

In [17]:
def mse(column_y_real,column_y_pred):
    n = len(column_y_real)
    numerador = 0.0
    
    for i in range(n):
        numerador += (column_y_real[i] - column_y_pred[i]) ** 2
    
    result = numerador / n
    
    return result

mse(y_test,column_y_pred)

46.32815261718384

In [18]:
x_train = train[colunas[0]]
x_test = test[colunas[0]]

best_column = ''
best_rmse = float('inf')

# Filter only continuous columns (exclude POINTID and categorical columns)
for column in colunas[1:]:  # Skip POINTID (first column)
    # Check if column is numeric (continuous)
    if train[column].dtype in ['float64', 'int64']:
        y_train = train[column]
        y_test = test[column]
        
        b1,b0 = calculo_b1_b0(x_train,y_train)
        column_y_pred = pred_test(x_test)
        
        rmse_value = rmse(y_test,column_y_pred)
        mse_value = mse(y_test,column_y_pred)
        
        if rmse_value < best_rmse:
            best_rmse = rmse_value
            best_column = column
        
        print(f"Coluna: {column}")
        print(f"RMSE: {rmse_value}")
        print(f"MSE: {mse_value}")
        print("---")
    else:
        print(f"Pulando coluna categórica: {column}")

print(f"Melhor coluna: {best_column} com RMSE: {best_rmse}")

Coluna: Bdfine (g cm-3)
RMSE: 0.2101397402681726
MSE: 0.04415871043997503
---
Coluna: SOCS (kg cm-2)
RMSE: 6.806478723773684
MSE: 46.32815261718384
---
Coluna: coarse_vol
RMSE: 0.04987954923264398
MSE: 0.0024879694316517547
---
Coluna: GPS_LAT
RMSE: 8.9816734249701
MSE: 80.67045751281415
---
Coluna: GPS_LONG
RMSE: 2.482074015914605
MSE: 6.160691420478455
---
Pulando coluna categórica: BDfine  method 
Melhor coluna: coarse_vol com RMSE: 0.04987954923264398
